In [ ]:
"""
Created on Fri Jan 27 15:18 2023

Look at the results of the test of the NN on Smith data

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import glob

In [ ]:
%matplotlib qt5

DEFINE OPTIONS

In [ ]:
TS_opt = 'extrap' # extrap, whole, thermocline
norm_method =  'std' # std, interquart, minmax
exp_name = 'newbasic2' #'allbutconstants' #'onlyTSdraftandslope' #'TSdraftbotandiceddandwcd' #'onlyTSisfdraft' #'TSdraftbotandiceddandwcdreldGL' #TSdraftslopereldGL

In [ ]:
#isf_dim = [10,11,12,13,18,22,23,25,30,31,33,39,40,42,43,44,45,47,51,55,58,61,65,66,69,70,71,73,75]

In [ ]:
nemo_run = 'bi646' #'bi646'

In [ ]:
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/'
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'
outputpath_melt = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'_CLASSIC/'
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'


RESULTS NN MODELS

In [ ]:
res_1D_mods_list = []
for mod_size in ['xsmall96', 'small', 'large']:
        outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'/'
        res_1D_all_list = []
        for yy in range(1980, 1980 + 60):
            res_1D_yy = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_1D_'+mod_size+'_'+exp_name+'_ensmean_'+TS_opt+'_norm'+norm_method+'_'+str(yy)+'_'+nemo_run+'.nc')
            res_1D_all_list.append(res_1D_yy.assign_coords({'time': yy}))
        res_1D_all_xr = xr.concat(res_1D_all_list, dim='time')
        res_1D_mods_list.append(res_1D_all_xr.assign_coords({'param': mod_size}))
res_1D_mods_all = xr.concat(res_1D_mods_list, dim='param')

In [ ]:
nn_melt_Gt_all = res_1D_mods_all['predicted_melt'].sel(metrics='Gt')
nn_melt_box1_all = res_1D_mods_all['predicted_melt'].sel(metrics='box1')

RESULTS CLASSIC PARAM

In [ ]:
#run_list = ['OPM031'] 
file_isf_list = []
target_melt_list = []
ds_melt_Gt_list = []
ds_melt_box1_list = []
ref_box1_list = []

for yy in tqdm(range(1980,1980 + 60)):
    
    # Param files
    melt_param_files = list(sorted(glob.glob(outputpath_melt+'eval_metrics_1D_*_oneFRIS_'+str(yy)+'.nc')))
    
    param_list = []
    for mfilename in melt_param_files:
        #paramname = mfilename[97:157].split('.')[0].split('_')[:-1]
        paramname = mfilename[87:157].split('.')[0].split('_')[:-2]
        #paramname = mfilename[105:157].split('.')[0].split('_')[:-1]
        mparam = '_'.join(paramname)
        param_list.append(mparam)
    
    ds_melt_param  = xr.open_mfdataset(melt_param_files, concat_dim='new_param', combine='nested', coords='minimal',compat='override')#, chunks={'x': chunksize, 'y': chunksize})
    if 'option' in ds_melt_param.coords:        
        ds_melt_param = ds_melt_param.drop('option')
    if 'calib' in ds_melt_param.coords: 
        ds_melt_param = ds_melt_param.drop('calib')
    if 'param' in ds_melt_param.coords: 
        ds_melt_param = ds_melt_param.drop('param')

    ds_melt_param = ds_melt_param.rename({'new_param': 'param'})
    ds_melt_param = ds_melt_param.assign_coords(param=param_list, time=yy)
    ds_melt_param_Gt = ds_melt_param['melt_1D_Gt_per_y'].sel(Nisf=ds_melt_param.Nisf)
    ds_melt_Gt_list.append(ds_melt_param_Gt)
    ds_melt_param_box1 = ds_melt_param['melt_1D_mean_myr_box1'].sel(Nisf=ds_melt_param.Nisf)
    ds_melt_box1_list.append(ds_melt_param_box1)
    
    # File for target
    NEMO_melt_rates_1D = xr.open_dataset(inputpath_data+'melt_rates_1D_NEMO_oneFRIS_'+str(yy)+'.nc')
    target_melt_Gt_yr = NEMO_melt_rates_1D['melt_Gt_per_y_tot'].sel(Nisf=ds_melt_param.Nisf)
    target_melt_list.append(target_melt_Gt_yr)
    NEMO_box1_myr = xr.open_dataset(inputpath_data+'melt_rates_box1_NEMO_oneFRIS_'+str(yy)+'.nc')
    ref_box1 = NEMO_box1_myr['mean_melt_box1_myr']
    ref_box1_list.append(ref_box1)
    
target_melt_all = xr.concat(target_melt_list, dim='time')
ref_box1_all = xr.concat(ref_box1_list, dim='time')
ds_melt_Gt_all = xr.concat(ds_melt_Gt_list, dim='time')
ds_melt_box1_all = xr.concat(ds_melt_box1_list, dim='time')


In [ ]:
nisf_of_int = res_1D_mods_all.Nisf.drop_sel(Nisf=4)

In [ ]:
param_melt_Gt_all = xr.concat([ds_melt_Gt_all,nn_melt_Gt_all], dim='param').load()
param_melt_box1_all = xr.concat([ds_melt_box1_all,nn_melt_box1_all], dim='param').load()

In [ ]:
diff_Gt_all_param = param_melt_Gt_all - target_melt_all


In [ ]:
RMSE_Gt_all_first30 = np.sqrt((diff_Gt_all_param.sel(time=range(1980,1980+30))**2).sel(Nisf=nisf_of_int).mean(['time','Nisf']))
RMSE_Gt_all_last30 = np.sqrt((diff_Gt_all_param.sel(time=range(1980+30,1980+60))**2).sel(Nisf=nisf_of_int).mean(['time','Nisf']))

In [ ]:
diff_box1_all_param_first30 = param_melt_box1_all.sel(time=range(1980,1980+30)).mean('time') - ref_box1_all.sel(time=range(1980,1980+30)).mean('time')
diff_box1_all_param_last30 = param_melt_box1_all.sel(time=range(1980+30,1980+60)).mean('time') - ref_box1_all.sel(time=range(1980+30,1980+60)).mean('time')

RMSE_box1_all_first30 = np.sqrt((diff_box1_all_param_first30**2).sel(Nisf=nisf_of_int).mean('Nisf'))
RMSE_box1_all_last30 = np.sqrt((diff_box1_all_param_last30**2).sel(Nisf=nisf_of_int).mean('Nisf'))

In [ ]:
param_list_of_int = ['boxes_3_pismyes_picopno', 
                     'lazero19_2', 
                     'quadratic_local_locslope',
                     'quadratic_local',
                    'xsmall96',
                    'small',
                    'large',
                    ]

In [ ]:
fig, axs = plt.subplots(1, 2,figsize=(8.24*1.25/1.5,8.24/2),sharey=True)

ccolors = ['steelblue','darkcyan','c','deepskyblue','skyblue','aquamarine','green','lightsteelblue','cyan','orange','magenta']

k = 0
#plt.figure()
for mparam in param_list_of_int[::-1]:

        
    axs[0].scatter(RMSE_Gt_all_first30.sel(param=mparam),mparam,marker='o',c='orange', s=50)
    axs[1].scatter(RMSE_box1_all_first30.sel(param=mparam),mparam,marker='o',c='orange', s=50)

    axs[0].scatter(RMSE_Gt_all_last30.sel(param=mparam),mparam,marker='o',c='red', s=50)
    axs[1].scatter(RMSE_box1_all_last30.sel(param=mparam),mparam,marker='o',c='red', s=50)

if nemo_run == 'bf663':
    axs[0].set_xlim(0,50)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,0.9)
elif nemo_run == 'bi646':
    axs[0].set_xlim(0,500)
    #axs[1].set_xlim(0,2.05)
    axs[1].set_xlim(0,0.9)

sns.despine()